But du notebook : recuperer et organiser les données afin de créer une carte interactive


# 1.Data loading


The data can be downloaded from the bixi montréal website.  
 https://bixi.com/fr/donnees-ouvertes  
you can download them by doing : 
 ```
 wget https://sitewebbixi.s3.amazonaws.com/uploads/docs/2021-stations-87d92f.zip 
 ```


In [56]:
import pandas as pd 
from datetime import datetime

In [57]:
deplacement = pd.read_csv('data/2021_donnees_ouvertes.csv')
deplacement.head()

,start_date,emplacement_pk_start,end_date,emplacement_pk_end,duration_sec,is_member
0,2021-06-29 17:46:28.653,10,2021-06-29 19:33:25.700,10,6417,0
1,2021-04-25 18:02:39.110,10,2021-04-25 18:13:26.139,188,647,0
2,2021-08-08 18:50:31.633,10,2021-08-08 18:58:57.585,778,505,1
3,2021-10-11 17:50:15.346,10,2021-10-11 18:25:31.726,504,2116,1
4,2021-09-10 15:51:58.050,10,2021-09-10 16:21:01.403,780,1743,1


In [ ]:
deplacement = deplacement.drop(columns=["end_date","duration_sec","is_member"])
deplacement_unique = deplacement.groupby(["emplacement_pk_start","emplacement_pk_end"]).count()
deplacement_unique = deplacement_unique.reset_index()
deplacement_unique.head()
deplacement_unique[deplacement_unique["emplacement_pk_start"]==deplacement_unique["emplacement_pk_end"]]["start_date"].sum()

253177

We success to import the 5 millions deplacements of Montrealers in 2021. 
Now we can import the stations because we have only the *pk* and not the exact emplacement

In [ ]:
stations = pd.read_csv('data/2021_stations.csv')
stations.loc[stations["latitude"]==-1.0, "latitude"]=45.49277
stations.loc[stations["longitude"]==-1.0, "longitude"]=-73.55641
stations.head()


,pk,name,latitude,longitude
0,10,Métro Angrignon (Lamont / des Trinitaires),45.446910,-73.603630
1,13,Métro de l'Église (Ross / de l'Église),45.462726,-73.565959
2,17,Tolhurst / Fleury,45.544149,-73.667520
3,21,St-Christophe / Cherrier,45.520290,-73.568119
4,31,Lespérance / de Rouen,45.538839,-73.552779


As you can see, we correct an error found earlier for the coordinates of the Peel/Smith station

We now perform a merge to obtain the geographical locations of each path

In [ ]:
start = deplacement_unique.merge(stations, left_on="emplacement_pk_start", right_on="pk")
start.rename({"latitude":"pickup_latitude", "longitude":"pickup_longitude", "start_date":"nb", "name":"pickup_name"},errors="raise",axis='columns',inplace=True)
start_end = start.merge(stations, left_on="emplacement_pk_end", right_on="pk")
start_end.rename({"latitude":"dropoff_latitude", "longitude":"dropoff_longitude"},errors="raise",axis='columns',inplace=True)
deplacement_final = start_end.drop(columns=["emplacement_pk_end", "emplacement_pk_start","pk_x","pk_y"])
deplacement_final.head()

,nb,pickup_name,pickup_latitude,pickup_longitude,name,dropoff_latitude,dropoff_longitude
0,763,Métro Angrignon (Lamont / des Trinitaires),45.446910,-73.603630,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363
1,46,Métro de l'Église (Ross / de l'Église),45.462726,-73.565959,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363
2,2,St-Christophe / Cherrier,45.520290,-73.568119,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363
3,10,Métro Vendôme (de Marlowe / de Maisonneuve),45.473868,-73.604538,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363
4,20,Métro Guy-Concordia (Guy / St-Catherine),45.495045,-73.577830,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363


In [ ]:
deplacement_final["name"]=deplacement_final["pickup_name"]+deplacement_final["name"]
deplacement_final.set_index("name",inplace=True)
deplacement_final.drop("pickup_name",axis=1)
deplacement_final.head()

,nb,pickup_name,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
name,,,,,,
Métro Angrignon (Lamont / des Trinitaires)Métro Angrignon (Lamont / des Trinitaires),763,Métro Angrignon (Lamont / des Trinitaires),45.446910,-73.603630,45.44691,-73.60363
Métro de l'Église (Ross / de l'Église)Métro Angrignon (Lamont / des Trinitaires),46,Métro de l'Église (Ross / de l'Église),45.462726,-73.565959,45.44691,-73.60363
St-Christophe / CherrierMétro Angrignon (Lamont / des Trinitaires),2,St-Christophe / Cherrier,45.520290,-73.568119,45.44691,-73.60363
Métro Vendôme (de Marlowe / de Maisonneuve)Métro Angrignon (Lamont / des Trinitaires),10,Métro Vendôme (de Marlowe / de Maisonneuve),45.473868,-73.604538,45.44691,-73.60363
Métro Guy-Concordia (Guy / St-Catherine)Métro Angrignon (Lamont / des Trinitaires),20,Métro Guy-Concordia (Guy / St-Catherine),45.495045,-73.577830,45.44691,-73.60363


In [ ]:
from pathlib import Path
filepath = Path('data/deplacement.csv')
# deplacement_final.to_csv(filepath)

A file containing all the trips is now savedA file containing all the trips is now saved and can be used in Keglergl.

In [ ]:
df=deplacement_final.groupby("pickup_name").count()
df

,nb,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
pickup_name,,,,,
10e avenue / Holt,500,500,500,500,500
10e avenue / Masson,566,566,566,566,566
12e avenue / St-Zotique,442,442,442,442,442
15e avenue / Masson,523,523,523,523,523
16e avenue / Beaubien,463,463,463,463,463
...,...,...,...,...,...
du Tricentenaire / Prince-Albert,28,28,28,28,28
la Fontaine / Alexandre-de-Sève,488,488,488,488,488
Édouard-Montpetit / de Stirling,571,571,571,571,571


Sum of deplacements by stations for 2021

In [124]:
count_stations=deplacement["emplacement_pk_start"].value_counts()
count_stations=pd.DataFrame({"station":count_stations.index,"nb_trajets":count_stations.values}).set_index("station")
count_stations=count_stations.merge(stations,left_on="station",right_on="pk")
count_stations=count_stations[["pk","name","latitude","longitude","nb_trajets"]]
count_stations.set_index("name",inplace=True)
count_stations.to_csv("data\\2021_count_stations.csv")
count_stations


,pk,latitude,longitude,nb_trajets
name,,,,
Métro Mont-Royal (Rivard / du Mont-Royal),256,45.524673,-73.582550,48310
du Mont-Royal / Clark,242,45.519410,-73.586850,36256
Marquette / du Mont-Royal,173,45.532290,-73.575440,36052
Parc Jeanne Mance (monument sir George-Etienne Cartier),330,45.514840,-73.584779,32159
Laurier / de Brébeuf,214,45.532514,-73.584811,31549
...,...,...,...,...
Parc St-Laurent (Dutrisac),1140,45.525914,-73.689111,9
Marmier,904,45.531007,-73.598799,7
René-Lévesque / 9e avenue,893,45.640739,-73.500826,7


Group deplacement by month

In [134]:
test=pd.to_datetime(deplacement["start_date"])
deplacement_with_time=deplacement.copy()
deplacement_with_time["start_date"]=test
deplacement_with_time.set_index("start_date",inplace=True)
deplacement_with_time


,emplacement_pk_start,end_date,emplacement_pk_end,duration_sec,is_member
start_date,,,,,
2021-06-29 17:46:28.653,10,2021-06-29 19:33:25.700,10,6417,0
2021-04-25 18:02:39.110,10,2021-04-25 18:13:26.139,188,647,0
2021-08-08 18:50:31.633,10,2021-08-08 18:58:57.585,778,505,1
2021-10-11 17:50:15.346,10,2021-10-11 18:25:31.726,504,2116,1
2021-09-10 15:51:58.050,10,2021-09-10 16:21:01.403,780,1743,1
...,...,...,...,...,...
2021-11-10 18:17:49.654,1140,2021-11-10 18:24:40.643,928,410,1
2021-11-11 14:55:04.039,1140,2021-11-11 14:59:41.446,1140,277,0
2021-11-12 14:50:15.504,1140,2021-11-12 15:05:37.379,1140,921,1


In [141]:
deplacement_with_time.loc["2021-06":"2021-06"]

C:\Users\Nono\AppData\Local\Temp\ipykernel_11548\2281455126.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  deplacement_with_time.loc["2021-06":"2021-06"]


,emplacement_pk_start,end_date,emplacement_pk_end,duration_sec,is_member
start_date,,,,,
2021-06-29 17:46:28.653,10,2021-06-29 19:33:25.700,10,6417,0
2021-06-23 19:21:47.265,10,2021-06-23 19:46:15.939,514,1468,0
2021-06-13 19:59:04.810,10,2021-06-13 20:24:34.247,206,1529,1
2021-06-01 17:13:37.536,10,2021-06-01 17:17:53.124,507,255,1
2021-06-06 18:58:36.573,10,2021-06-06 19:37:39.973,10,2343,1
...,...,...,...,...,...
2021-06-29 16:42:09.384,1025,2021-06-29 16:50:58.977,265,529,1
2021-06-29 17:27:37.862,1025,2021-06-29 17:31:44.037,265,246,1
2021-06-30 18:55:27.037,1025,2021-06-30 19:06:59.314,21,692,1
